<center><h1>🔥  FHIR Kindling</h1></center>

Python fhir client library for easier and safer interactions with FHIR servers and resources


## Features

- Create, Read, Update and Delete resources using a server's REST API
- Resource validation
- Transfer resources between fhir servers
- CSV/Dataframe serialization for resources
- Synthetic data generation and upload


## Installation

Install the latest published version from pypi:
```bash
pip install --user fhir-kindling
```
or install the newest version directly from github:
```bash
pip install --user git+https://github.com/migraf/fhir-kindling.git
```


In [ ]:
!pip install --upgrade fhir-kindling

<center><h2>👨‍💻   How To</h2></center>

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
from fhir_kindling import FhirServer

_ = load_dotenv(find_dotenv())

In [2]:

fhir_api = "https://demo.personalhealthtrain.de/demo-fhir-3"
username = os.getenv("DEMO_USER")
password = os.getenv("DEMO_PW")
server = FhirServer(
    api_address=fhir_api,
    username=username,
    password=password,
)

## Query for resources

Query the server with the `query()` method of the server class.

There are three ways to define a query:
- Iteratively build the query on a resource using methods like `where()`, `include()`, `has()`
- Use a `query_string` to define the query ie `Patient?_id=123"`
- Pass a `FHIRQueryParameters` object to the query method

## Iteratively building a query

Start building a query by selecting the base resource first

In [3]:
query = server.query("Patient")
query.query_url

'https://demo.personalhealthtrain.de/demo-fhir-3/Patient?&_count=5000&_format=json'

### Querying the server
the query is executed against the server using one of the methods `all()`, `first()`, `limit()`

In [4]:
response = query.all()
response

<QueryResponse(resource=Patient, n=271)>

In [5]:
response = query.limit(5)
response

<QueryResponse(resource=Patient, n=5)>

Accessing the resources in a `QueryResponse` object.

In [6]:
response.resources[0]

Patient(resource_type='Patient', fhir_comments=None, id='1', implicitRules=None, implicitRules__ext=None, language=None, language__ext=None, meta=Meta(resource_type='Meta', fhir_comments=None, extension=None, id=None, lastUpdated=datetime.datetime(2022, 6, 8, 10, 36, 17, 669000, tzinfo=datetime.timezone.utc), lastUpdated__ext=None, profile=None, profile__ext=None, security=None, source='#SCHsV8ok4VcRcOjM', source__ext=None, tag=None, versionId='1', versionId__ext=None), contained=None, extension=None, modifierExtension=None, text=Narrative(resource_type='Narrative', fhir_comments=None, extension=None, id=None, div='<div xmlns="http://www.w3.org/1999/xhtml"><div class="hapiHeaderText">Duncann <b>NOVARA </b></div><table class="hapiPropertyTable"><tbody><tr><td>Date of birth</td><td><span>08 August 1968</span></td></tr></tbody></table></div>', div__ext=None, status='generated', status__ext=None), active=None, active__ext=None, address=None, birthDate=datetime.date(1968, 8, 8), birthDate__

### Adding filter conditions

Filter parameters are added on the fields of the base resource using the `where()` method.

In [7]:
query_2 = server.query("Patient").where("birthdate", "lt", "2000-01-01")
query_2.query_url

'https://demo.personalhealthtrain.de/demo-fhir-3/Patient?birthdate=lt2000-01-01&_count=5000&_format=json'

In [8]:
query_2.all()

<QueryResponse(resource=Patient, n=254)>

### Including related resources

In [9]:
query_3 = query_2.include(resource="Condition", reference_param="subject", reverse=True)
query_3.query_url

'https://demo.personalhealthtrain.de/demo-fhir-3/Patient?birthdate=lt2000-01-01&_revinclude=Condition:subject&_count=5000&_format=json'

In [10]:
resp = query_3.all()
resp

<QueryResponse(resource=Patient, format=json, included_resources=['Condition'])>

## Working with the response

The response to the query is a `QueryResponse` object.

- The `resources` attribute contains a list of resources of the base resource type returned by the query
- The `included_resources` attribute contains a list of included resources. Each entry in the


In [11]:
[resource.resource_type for resource in resp.included_resources]

['Condition']

In [12]:
resp.included_resources[0].resources[0]

Condition(resource_type='Condition', fhir_comments=None, id='513', implicitRules=None, implicitRules__ext=None, language=None, language__ext=None, meta=Meta(resource_type='Meta', fhir_comments=None, extension=None, id=None, lastUpdated=datetime.datetime(2022, 7, 26, 14, 18, 29, 393000, tzinfo=datetime.timezone.utc), lastUpdated__ext=None, profile=None, profile__ext=None, security=None, source='#8wHK4OI5kklZAuKD', source__ext=None, tag=None, versionId='1', versionId__ext=None), contained=None, extension=None, modifierExtension=None, text=None, abatementAge=None, abatementDateTime=None, abatementDateTime__ext=None, abatementPeriod=None, abatementRange=None, abatementString=None, abatementString__ext=None, asserter=None, bodySite=None, category=None, clinicalStatus=None, code=CodeableConcept(resource_type='CodeableConcept', fhir_comments=None, extension=None, id=None, coding=[Coding(resource_type='Coding', fhir_comments=None, extension=None, id=None, code='RA01.0', code__ext=None, display

### Saving the response

Responses can be saved to a file using the `save()` method of the `QueryResponse` class.
Supported formats are `json`, `xml` (if the query was executed with `xml` format) and `csv`.

In [13]:
path = os.path.join(os.getcwd(), "query_response.json")
resp.save(file_path=path)

In [14]:
with open(path, "r") as f:
    print("".join(f.readlines()[:8]))

{
  "resourceType": "Bundle",
  "id": "4fa17c4e-d005-4485-920f-85dbe2a38f05",
  "meta": {
    "lastUpdated": "2022-07-26T15:07:55.371000+00:00"
  },
  "type": "searchset",
  "total": 254,



### Serializing resources into a pandas dataframe

A response (or any bundle) can be serialized into pandas dataframes.
If the response contains resources of different types, the resources are serialized into separate dataframes for each type.

In [15]:
dfs = resp.to_dfs()

In [17]:
dfs[0].head()

,resourceType,id,meta_versionId,meta_lastUpdated,meta_source,text_status,text_div,name_0_family,name_0_given_0,gender,birthDate
0,Patient,100,1,2022-06-08 10:36:17.669000+00:00,#SCHsV8ok4VcRcOjM,generated,"<div xmlns=""http://www.w3.org/1999/xhtml""><div...",Aiola,Tyniqua,male,1921-09-30
1,Patient,391,1,2022-07-26 14:18:27.623000+00:00,#QmJ643x11dMzK1oE,generated,"<div xmlns=""http://www.w3.org/1999/xhtml""><div...",Graveran,Elliekate,female,1921-11-04
2,Patient,407,1,2022-07-26 14:18:27.623000+00:00,#QmJ643x11dMzK1oE,generated,"<div xmlns=""http://www.w3.org/1999/xhtml""><div...",Anuska,Zamoni,female,1921-11-30
3,Patient,39,1,2022-06-08 10:36:17.669000+00:00,#SCHsV8ok4VcRcOjM,generated,"<div xmlns=""http://www.w3.org/1999/xhtml""><div...",Ramdeo,Corzine,male,1921-12-27
4,Patient,94,1,2022-06-08 10:36:17.669000+00:00,#SCHsV8ok4VcRcOjM,generated,"<div xmlns=""http://www.w3.org/1999/xhtml""><div...",Mullaly,Treye,male,1922-08-14


In [18]:
dfs[1].head()

,resourceType,id,meta_versionId,meta_lastUpdated,meta_source,code_coding_0_system,code_coding_0_code,code_coding_0_display,code_text,subject_reference,clinicalStatus_coding_0_system,clinicalStatus_coding_0_code,clinicalStatus_coding_0_display
0,Condition,513,1,2022-07-26 14:18:29.393000+00:00,#8wHK4OI5kklZAuKD,http://id.who.int/icd/release/11/mms,RA01.0,"COVID-19, virus identified",COVID-19,Patient/413,NaN,NaN,NaN
1,Condition,514,1,2022-07-26 14:18:29.393000+00:00,#8wHK4OI5kklZAuKD,http://id.who.int/icd/release/11/mms,RA01.0,"COVID-19, virus identified",COVID-19,Patient/414,NaN,NaN,NaN
2,Condition,515,1,2022-07-26 14:18:29.393000+00:00,#8wHK4OI5kklZAuKD,http://id.who.int/icd/release/11/mms,RA01.0,"COVID-19, virus identified",COVID-19,Patient/415,NaN,NaN,NaN
3,Condition,516,1,2022-07-26 14:18:29.393000+00:00,#8wHK4OI5kklZAuKD,http://id.who.int/icd/release/11/mms,RA01.0,"COVID-19, virus identified",COVID-19,Patient/416,NaN,NaN,NaN
4,Condition,517,1,2022-07-26 14:18:29.393000+00:00,#8wHK4OI5kklZAuKD,http://id.who.int/icd/release/11/mms,RA01.0,"COVID-19, virus identified",COVID-19,Patient/417,NaN,NaN,NaN


### Converting a list of resources to a dataframe

Any list of resources (pydantic models or dicts) can be converted to a dataframe using the `flattten()` method.

In [19]:
from fhir_kindling.serde import flatten_resources

# get a list of patient resources
patients = server.query("Patient").limit(100).resources

In [20]:
flatten_resources(patients)

,resourceType,id,meta_versionId,meta_lastUpdated,meta_source,text_status,text_div,name_0_family,name_0_given_0,gender,birthDate
0,Patient,1,1,2022-06-08 10:36:17.669000+00:00,#SCHsV8ok4VcRcOjM,generated,"<div xmlns=""http://www.w3.org/1999/xhtml""><div...",Novara,Duncann,male,1968-08-08
1,Patient,2,1,2022-06-08 10:36:17.669000+00:00,#SCHsV8ok4VcRcOjM,generated,"<div xmlns=""http://www.w3.org/1999/xhtml""><div...",Weisman,Damiso,other,1981-01-26
2,Patient,3,1,2022-06-08 10:36:17.669000+00:00,#SCHsV8ok4VcRcOjM,generated,"<div xmlns=""http://www.w3.org/1999/xhtml""><div...",Newball,Kimbrick,male,1994-12-30
3,Patient,4,1,2022-06-08 10:36:17.669000+00:00,#SCHsV8ok4VcRcOjM,generated,"<div xmlns=""http://www.w3.org/1999/xhtml""><div...",Gargan,Chanae,female,1969-12-07
4,Patient,5,1,2022-06-08 10:36:17.669000+00:00,#SCHsV8ok4VcRcOjM,generated,"<div xmlns=""http://www.w3.org/1999/xhtml""><div...",Aromin,Kallysta,male,1943-11-06
...,...,...,...,...,...,...,...,...,...,...,...
95,Patient,96,1,2022-06-08 10:36:17.669000+00:00,#SCHsV8ok4VcRcOjM,generated,"<div xmlns=""http://www.w3.org/1999/xhtml""><div...",Balitas,Eligijus,female,1989-11-05
96,Patient,97,1,2022-06-08 10:36:17.669000+00:00,#SCHsV8ok4VcRcOjM,generated,"<div xmlns=""http://www.w3.org/1999/xhtml""><div...",Kliskey,Quierra,other,2000-11-14
97,Patient,98,1,2022-06-08 10:36:17.669000+00:00,#SCHsV8ok4VcRcOjM,generated,"<div xmlns=""http://www.w3.org/1999/xhtml""><div...",Arya,Welsey,female,1981-08-26
98,Patient,99,1,2022-06-08 10:36:17.669000+00:00,#SCHsV8ok4VcRcOjM,generated,"<div xmlns=""http://www.w3.org/1999/xhtml""><div...",Zaxas,Juawan,male,1951-03-10


## Generating synthetic data

Generate complex synthetic data sets using dataset and resource generator functions.
Interdependencies between resources and the likelihood of a resource being generated can be defined.


This example will generate a dataset with:
- Patients
- with Covid-19 conditions
- a certain likelihood of being vaccinated.

Start by importing and defining some constants i.e. Codes for the condition and the vaccination.

In [ ]:
from fhir.resources.codeableconcept import CodeableConcept
from fhir.resources.coding import Coding

from fhir_kindling.generators.patient import PatientGenerator
from fhir_kindling.generators.resource_generator import ResourceGenerator, GeneratorParameters, FieldValue
from fhir_kindling.generators.field_generator import FieldGenerator
from fhir_kindling.generators.dataset import DatasetGenerator
from fhir_kindling.fhir_query.query_parameters import QueryOperators

import pendulum

covid_code = CodeableConcept(
    coding=[
        Coding(
            system="http://id.who.int/icd/release/11/mms",
            code="RA01.0",
            display="COVID-19, virus identified"
        )
    ],
    text="COVID-19"
)

vaccination_code = CodeableConcept(
    coding=[
        Coding(
            system="http://id.who.int/icd/release/11/mms",
            code="XM0GQ8",
            display="COVID-19 vaccine, RNA based"
        )
    ],
    text="COVID vaccination"
)


Configure the data set generator and subgenerators

In [ ]:
count = 100
dataset_generator = DatasetGenerator("Patient", n=count)

covid_params = GeneratorParameters(
    field_values=[
        FieldValue(field="code", value=covid_code),
    ]
)

covid_generator = ResourceGenerator("Condition", generator_parameters=covid_params)
# add covid conditions to patients
dataset_generator.add_resource(covid_generator, name="covid")

# patients, patient_ids = PatientGenerator(n=count, generate_ids=True).generate(references=True)

vaccination_date_generator = FieldGenerator(
    field="occurrenceDateTime",
    generator_function=lambda: pendulum.now().to_date_string()
)

first_vax_params = GeneratorParameters(
    field_values=[
        FieldValue(field="vaccineCode", value=vaccination_code),
        FieldValue(field="status", value="completed"),
    ],
    field_generators=[
        vaccination_date_generator
    ]
)
vaccination_generator = ResourceGenerator("Immunization", generator_parameters=first_vax_params)
dataset_generator.add_resource(vaccination_generator, name="first_vaccination", likelihood=0.8)

dataset = dataset_generator.generate(ids=True)

dataset.upload(server)

Check if our server now has covid patients

In [ ]:
covid_query = server.query("Patient").has(
    resource="Condition",
    search_param="code",
    operator=QueryOperators.eq,
    value="RA01.0",
    reference_param="subject",
).include(
    resource="Condition",
    reference_param="subject",
    reverse=True
)

covid_response = covid_query.all()
covid_response

## Transferring resources from one server to another

Use the `transfer()` function on a server object to transfer resources from one server to another while keeping referential integrity and using server assigned ids.
The transfer is a three-step process:
1. Analyze the resources to be transferred and build a DAG modeling the references
2. Obtain any missing resources that are referenced from the source server
3. Upload the resources to the target server based on the reference DAG

In [ ]:
# define a new server
transfer_api_url = "https://demo.personalhealthtrain.de/demo-fhir-4"
transfer_server = FhirServer(api_address=transfer_api_url, username=username, password=password)

server.transfer(transfer_server, covid_response)

In [ ]:
transfer_covid_query = transfer_server.query("Patient").has(
    resource="Condition",
    search_param="code",
    operator=QueryOperators.eq,
    value="RA01.0",
    reference_param="subject",
)
transfer_covid_query.all()

In [ ]:
<center><h2>Questions?</h2></center>
<center><h3>What feature would you like to have in a FHIR library?</h3></center>

## Transferring resources from one server to another

Use the `transfer()` function on a server object to transfer resources from one server to another while keeping referential integrity and using server assigned ids.
The transfer is a three-step process:
1. Analyze the resources to be transferred and build a DAG modeling the references
2. Obtain any missing resources that are referenced from the source server
3. Upload the resources to the target server based on the reference DAG

In [29]:
# define a new server
transfer_api_url = "https://demo.personalhealthtrain.de/demo-fhir-4"
transfer_server = FhirServer(api_address=transfer_api_url, username=username, password=password)

server.transfer(transfer_server, covid_response)

<TransferResponse(origin_server=https://demo.personalhealthtrain.de/demo-fhir-3, destination_server=https://demo.personalhealthtrain.de/demo-fhir-4, query_parameters=resource='Patient' resource_parameters=None include_parameters=[IncludeParameter(resource='Condition', search_param='subject', target=None, reverse=True, iterate=False)] has_parameters=[ReverseChainParameter(operator=<QueryOperators.eq: 'eq'>, value='RA01.0', resource='Condition', reference_param='subject', search_param='code')], create_responses=<ResourceCreateResponse(resource_id=643, location=Patient/643, version=None)>...<ResourceCreateResponse(resource_id=842, location=Patient/842, version=None)>)

In [31]:
transfer_covid_query = transfer_server.query("Patient").has(
    resource="Condition",
    search_param="code",
    operator=QueryOperators.eq,
    value="RA01.0",
    reference_param="subject",
)
transfer_covid_query.all()

<QueryResponse(resource=Patient, n=0)>

<center><h2>Questions?</h2></center>
<center><h3>What feature would you like to have in a FHIR library?</h3></center>